In [3]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
from collections import defaultdict as dd
from tqdm.notebook import tqdm
import re

In [21]:
# функция assert для проверки доступа к сайту
def check (link):
    response=requests.get(link)
    assert str(response) == '<Response [200]>'
    return 'доступ к сайту есть'

# основная страница, с которой будем брать информацию
url ='https://habr.com/ru/flows/develop/page1/'
response=requests.get(url)
s=response.content
soup = bs(s,'html.parser')

check(url)

'доступ к сайту есть'

In [3]:
item = soup.find_all('div', {'class' : "tm-article-snippet tm-article-snippet"})
item_name = soup.find_all('h2', {'class' : "tm-title tm-title_h2"})
# проверю соответствие на 1 странице
len(item_name)

20

In [4]:
# название статьи
item[0].find('h2', {'class' : "tm-title tm-title_h2"}).text

'Мультяшная графика продаётся лучше реалистичной. Так ли это на самом деле?'

In [5]:
# ссылка статьи после основной 
item_name[0].a.get('href')

'/ru/articles/734892/'

In [6]:
d = dd(lambda:{})
for page in range(1, 51):
    #для каждой страницы заменяю номер в ссылке
    url = f'https://habr.com/ru/flows/develop/page{page}/'
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    #нахожу все товары на странице
    items_on_one_page = soup.find_all('div', {'class' : "tm-article-snippet tm-article-snippet"})
    #для каждой статьи на странице ищу ссылку и название, добавляю в словарь
    for item in items_on_one_page:
        #href = item.a.get('href')
        name = item.find('h2', {'class' : "tm-title tm-title_h2"}).text
        href = item.find('h2', {'class' : "tm-title tm-title_h2"}).a.get('href')
        d[href] = {'item name': name}
#вывожу в удобной форме через pandas
pd.DataFrame(d).T

,item name
/ru/articles/734892/,Мультяшная графика продаётся лучше реалистично...
/ru/articles/734890/,Picture in Picture Mode в android. Показываем ...
/ru/articles/734888/,"Много нас, а GPU один. Как делиться?"
/ru/articles/734830/,Ближайшие бесплатные мероприятия по разработке...
/ru/companies/selectel/articles/734860/,Одноплатник Orange Pi 5 Plus: характеристики и...
...,...
/ru/articles/728896/,"Java на стероидах, или опыт работы с Jmix"
/ru/companies/sportmaster_lab/articles/728880/,"«Чистый» код, ужасная производительность"
/ru/articles/723668/,PostgreSQL под капотом. Часть 5. Простой SELEC...
/ru/articles/728870/,Чат в терминале на Rust


Теперь возьмем одну статью и выделим все признаки для нее в словарь

*Там в коде и в выводе различаются некоторые данные, потому что они успели поменяться за то время, пока мы чинили все, ну это не играет никакой роли, очень лень переделывать, поэтому будут небольшие расхождения

In [7]:
#проход по странице конкретной статьи /ru/articles/734556/
url = 'https://habr.com/ru/articles/734556/'
response = requests.get(url)
soup = bs(response.content, 'html.parser')
di={}

# ищем Время чтения в коде
#<span class="tm-article-reading-time__label"> 3 мин </span>
read_time=soup.find('span', {'class': re.compile(r'tm-article-reading-time__label')}).text.strip()
di['Время чтения']=read_time #добавляем в словарь
read_time

'3 мин'

In [8]:
#ищем уровень сложности статьи 
#<span class="tm-article-complexity__label">Простой</span>

complexity=soup.find('span', {'class': 'tm-article-complexity__label'}).text.strip()
di['Сложность']=complexity #добавляем в словарь
complexity

'Простой'

In [9]:
#ищем количество просмотров статьи 
#<span class="tm-icon-counter__value">3.2K</span>
views=soup.find('span', {'class': 'tm-icon-counter__value'}).text.strip()
di['Количество просмотров']=views #добавляем в словарь
views

'6.6K'

In [10]:
#ищем количество сохранений
#<span title="Количество пользователей, добавивших публикацию в закладки" class="bookmarks-button__counter"> 22 </span>
saved=soup.find('span', {'class': 'bookmarks-button__counter'}).text.strip()
di['В закладках']=saved #добавляем в словарь
saved

'34'

In [11]:
#ищем количество комментариев
#<span class="tm-article-comments-counter-link__value">6  </span>
comments=soup.find('span', {'class': 'tm-article-comments-counter-link__value'}).text.strip()
di['Комментарии']=comments #добавляем в словарь
comments

'11'

In [12]:
# имя пользователя
#<a href="/ru/users/gag_fenix/" 
#class="tm-user-card__nickname tm-user-card__nickname tm-user-card__nickname_variant-article">
         # @gag_fenix </a>
author_name=soup.find('a', {'class': 'tm-user-card__nickname tm-user-card__nickname tm-user-card__nickname_variant-article'}).text.strip()
di['Автор']=author_name #добавляем в словарь
author_name

'@gag_fenix'

In [13]:
#<span class="tm-votes-lever__score-counter tm-votes-lever__score-counter tm-votes-lever__score-counter_rating">
      #  24
     # </span>
author_rate=soup.find('span', {'class': 'tm-votes-lever__score-counter tm-votes-lever__score-counter tm-votes-lever__score-counter_rating'}).text.strip()
di['Рейтинг автора']=author_rate #добавляем в словарь
author_rate

'31.1'

In [16]:
# время публикации
#<time datetime="2023-05-12T11:26:10.000Z" title="2023-05-12, 14:26">9 часов назад</time>
publish_time=soup.time.get('title')
di['Время публикации']=publish_time #добавляем в словарь
publish_time

'2023-05-12, 14:26'

Поняли как вытаскивать признаки, теперь можно сделать это для всех объектов

In [17]:
# функция для каждой статьи
def parse_one_item(soup):
    di={}
    read_time=soup.find('span', {'class': re.compile(r'tm-article-reading-time__label')}).text.strip()
    di['Время чтения']=read_time
    if (soup.find('span', {'class': 'tm-article-complexity__label'})) is not None:
        complexity=soup.find('span', {'class': 'tm-article-complexity__label'}).text.strip()
        di['Сложность']=complexity
    else:
        di['Сложность']=None
    views=soup.find('span', {'class': 'tm-icon-counter__value'}).text.strip()
    di['Количество просмотров']=views
    saved=soup.find('span', {'class': 'bookmarks-button__counter'}).text.strip()
    di['В закладках']=saved
    comments=soup.find('span', {'class': 'tm-article-comments-counter-link__value'}).text.strip()
    di['Комментарии']=comments
    if soup.find('a', {'class': 'tm-user-card__nickname tm-user-card__nickname tm-user-card__nickname_variant-article'})is not None:
        author_name=soup.find('a', {'class': 'tm-user-card__nickname tm-user-card__nickname tm-user-card__nickname_variant-article'}).text.strip()
        di['Автор']=author_name
    else:
        di['Автор']=None
    if soup.find('span', {'class': 'tm-votes-lever__score-counter tm-votes-lever__score-counter tm-votes-lever__score-counter_rating'}) is not None:
        author_rate=soup.find('span', {'class': 'tm-votes-lever__score-counter tm-votes-lever__score-counter tm-votes-lever__score-counter_rating'}).text.strip()
        di['Рейтинг автора']=author_rate
    else:
        di['Рейтинг автора']=None
    publish_time=soup.time.get('title')
    di['Время публикации']=publish_time
    return di

In [19]:
# проверим работу функции для одной рандомной статьи
response = requests.get('https://habr.com/ru/articles/734806/')
soup = bs(response.content, 'html.parser')
parse_one_item(soup)

{'Время чтения': '3 мин',
 'Сложность': 'Простой',
 'Количество просмотров': '2K',
 'В закладках': '19',
 'Комментарии': '0',
 'Автор': '@wiseplat',
 'Рейтинг автора': '-6.9',
 'Время публикации': '2023-05-12, 21:42'}

In [20]:
# берем базовую часть ссылки, которая есть у каждой статьи
base_url = 'https://habr.com'
for item_url in tqdm(d): # в цикле для ссылки каждой статьи
    url = base_url+item_url # прибавляем базовую часть ссылки к конкретной, чтобы попасть на конкретную страницу 
    response = requests.get(url)
    soup = bs(response.content, 'html.parser')
    results = parse_one_item(soup) #применяем к конкретной статье функцию
    d[item_url].update(results) # добавляем подробную информацию о статье в существующий словарь
pd.DataFrame(d).T.to_csv('habr_data.csv') #создаем таблицу

  0%|          | 0/1000 [00:00<?, ?it/s]

Ура! Мы собрали все данные, теперь будет работать с файлом habr_data.csv